<small><font color=gray>Notebook author: <a href="https://www.linkedin.com/in/olegmelnikov/" target="_blank">Oleg Melnikov</a> ©2021 onwards</font></small><hr style="margin:0;background-color:silver">

**[<font size=6>🍿Netflix</font>](https://www.kaggle.com/c/19jun23jh-netflix/rules)**. [**Instructions**](https://colab.research.google.com/drive/1riOGrE_Fv-yfIbM5V4pgJx4DWcd92cZr#scrollTo=ITaPDPIQEgXV) for running Colabs.

<small>**(Optional) CONSENT.** <mark>[ X ]</mark> We consent to sharing our Colab (after the assignment ends) with other students/instructors for educational purposes. We understand that sharing is optional and this decision will not affect our grade in any way. <font color=gray><i>(If ok with sharing your Colab for educational purposes, leave "X" in the check box.)</i></font></small>

In [ ]:
from google.colab import drive; drive.mount('/content/drive')   # OK to enable, if your kaggle.json is stored in Google Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q --upgrade imgaug==0.2.5 > log.txt  # avoids warning/conflict with scipy upgrade
!pip install -q --upgrade scipy >>log.txt  # upgrades 1.4 to 1.8

In [ ]:
!pip -q install --upgrade --force-reinstall --no-deps kaggle > log  # upgrade kaggle package (to avoid a warning)
!mkdir -p ~/.kaggle                                           # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
!cp kaggle.json ~/.kaggle/kaggle.json > log                   # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                              # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v 19jun23jh-netflix        # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download >> log                          # download competition dataset as a zip file
!unzip -o *.zip >> log                                        # Kaggle dataset is copied as a single file and needs to be unzipped.
!kaggle competitions leaderboard --show                       # print public leaderboard

cp: cannot stat 'kaggle.json': No such file or directory
- competition is now set to: 19jun23jh-netflix
Using competition: 19jun23jh-netflix
  teamId  teamName        submissionDate       score    
--------  --------------  -------------------  -------  
10602951  Group 9         2023-07-02 00:12:04  0.42990  
10602783  Group 7         2023-07-02 22:03:13  0.42490  
10606288  Group 3         2023-07-02 06:29:58  0.42270  
10609906  Group 10        2023-07-02 17:27:13  0.42150  
10607335  Group 4         2023-06-30 07:10:08  0.42020  
10580078  Group 12        2023-07-02 23:03:13  0.41960  
10602607  Team 13         2023-07-01 18:51:11  0.41900  
10580383  Group 5         2023-07-02 14:39:06  0.41870  
10607564  Team 14         2023-07-02 17:19:16  0.41800  
10615163  Group 2         2023-07-02 16:20:24  0.41690  
10603056  Group 6         2023-07-02 00:45:44  0.41670  
10592821  Group 1         2023-07-03 00:01:07  0.41660  
10601940  Team 8          2023-07-02 02:34:59  0.41600  
1061

In [ ]:
%%time
%%capture
%reset -f
!cp metadata.json METADATA
!pip -q install scikit-surprise
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all"
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, seaborn as sns, os, tqdm, re, sys, cv2, skimage, plotly.express as px, surprise
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import TruncatedSVD
from scipy.sparse.linalg import svds
ToCSV = lambda df, fname: df.round(2).to_csv(f'{fname}.csv', index_label='id') # rounds values to 2 decimals

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

np.set_printoptions(linewidth=1000, precision=2, edgeitems=20, suppress=True)
pd.set_option('display.max_columns', 20, 'display.precision', 2, 'display.max_rows', 4)

CPU times: user 1.42 s, sys: 25 ms, total: 1.44 s
Wall time: 9.37 s


The matrix below contains ratings that users (user id or `uid`) have given to the movies (movie ID or `mid`). It is 85% blank, but the original dataset was 99% sparse (with ~1 rating per user) [&#127910;](https://youtu.be/ImpV70uLxyw). Your task is not to fill the whole matrix, but to impute `NaN` values in `XY_Netflix.csv`, which is an equivalent representation of `XY_Netflix_matrix.csv`. This is a collaborative filtering problem, where observations and features have some implicit relation. Recall that typically we expect independent observations in a sample (unless we have a time series or video of image frames, etc.). Here our model needs to learn and leverage the dependencies from observed movie ratings. Clustering techniques might be more intuitive here. [Truncated SVD](https://scikit-learn.org/stable/modules/decomposition.html#lsa) is also popular. Supervised models can be used as well, if we can build features describing our users (which we would treat as observations). Note that we have no other information on users except what they have ranked (which might suggest the cluster they belong to, which might offer unobserved (latent) features). [&#127910;](https://youtu.be/8wLKuscyO9I) Since users relate to other users through the movies they ranked, this can also be presented as a [bipartite graph](https://en.wikipedia.org/wiki/Bipartite_graph).

In [ ]:
dfMtx = pd.read_csv('XY_Netflix_matrix.csv', low_memory=False); dfMtx

,1,2,3,4,5,6,7,8,9,10,...,371,372,373,374,375,376,377,378,379,380
0,NaN,NaN,2.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5.0,NaN,5.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128697,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128698,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
f = px.imshow(dfMtx.iloc[:2000,:].T, color_continuous_scale='RdBu_r', aspect='auto');  # movies by users (transposed for visual convenience)
f.update_layout(margin=dict(l=0,r=0,b=0,t=0))

In [ ]:
dfMtx.info()   # note the datatypes and any missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128699 entries, 0 to 128698
Columns: 380 entries, 1 to 380
dtypes: float64(380)
memory usage: 373.1 MB


The dataframe below is another representation of the matrix form above. It lists the `uid`$\times$`mid`locations of the values you need to predict (those with `NaN`) and all other non-zero values in the matrix above.

In [ ]:
df = pd.read_csv('XY_Netflix.csv', low_memory=False); df  # user ID, movie ID, rating. NaN=requires predictions

,uid,mid,rtg
0,1,1,NaN
1,2,2,NaN
...,...,...,...
7402070,49692,126,4.0
7402071,9783,37,1.0


In [ ]:
df.rtg.fillna(0).astype(int).value_counts().to_frame().T   # rating frequencies. 0 represents NaN

,4,3,5,2,1,0
rtg,2451087,2120585,1628122,842056,340222,20000


In [ ]:
dfMov = pd.read_csv('movies.csv'); dfMov.head(2)   # load movie attributes

,mid,Year,Name,voteAvg,voteCnt,popularity,duration,income,budget,releaseDate,...,origLang,spokenLang,ttl,origTtl,URL,kwds,desc,prodComp,prodCtry,tagline
0,1,2004,The Village,6.2,1071,27.49,108.0,256697520,60000000,7/30/04,...,en,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",The Village,The Village,NaN,"[{""id"": 1328, ""name"": ""secret""}, {""id"": 5774, ...",When a willful young man tries to venture beyo...,"[{""name"": ""Scott Rudin Productions"", ""id"": 258...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",There is no turning back
1,2,1997,Liar Liar,6.4,1424,26.72,86.0,181000000,45000000,3/21/97,...,en,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Liar Liar,Liar Liar,NaN,"[{""id"": 387, ""name"": ""california""}, {""id"": 102...",Fletcher Reede is a fast-talking attorney and ...,"[{""name"": ""Imagine Entertainment"", ""id"": 23}, ...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",Trust me.


In [ ]:
tmr = Timer()

⏳ started. You have 60 sec. Good luck!


<hr color=red>

<font size=5>⏳</font> <strong><font color=orange size=5>Your Code, Documentation, Ideas and Timer - All Start Here...</font></strong>

**Student's Section** (between ⏳ symbols): add your code and documentation here.

### Group Members

* Amin Diab
* Neil Joshi
* Alejandro Salamanca
* Marc Estafanous

## **Task 1. Preprocessing Pipeline**

**1. Explain elements of your preprocessing pipeline i.e. feature engineering, subsampling, clustering, dimensionality reduction, etc.**

In this assignment, multiple approaches were attempted, the elements considered in the final preprocessing pipeline are listed below. The approaches attempted but left outside the final pipeline are shown in the section pertinent to what was attempted but was not adopted:

- Data subsampling
- Data shuffling
- Feature assessment

**2. Why did you choose these elements (perhaps something in EDA or prior experience lead you to these)**

- **For subsampling**: We noted the relatively high dimensional nature of the data, which coupled with the time limit of the running of the model encouraged us to pay attention to the number of samples used for training and testing. In the early stages of data exploration, we tried multiple sizes of dataset to be used as inputs, ranging from 500,000 to 4,000,000 as well as an approach which included identifying the set of points that pertain to user ids known to require predictions as per code in the colab file. We noted that on multiple modeling approaches, keeping the number of observations below approximately 2,500,000 enabled the model to abide by the time limit. We also noted that the performance improves when we chose to include a dataset that is composed of the ids of the users known to require predictions.

- **For data shuffling**: In the early stages of the model development, it was believed that order ought not to influence the performance of the models and we reasoned to remove any impact of order in the set, therefore we chose to shuffle the data.

- **For feature selection or assessment**: We noted from the literature including the videos provided in the colab file that some movies may polirize opinion regarding ratings (movies such as Napoleon Dynamite). We also noted that the model performance improved when those movies were removed from the set, yet we reasoned that removing the movies from the set may risk the model not learning about *any* preferences or ratings for those movies and the observed performance improvement may not hold if the movie appeared in the test set encountered, for this reason we chose to keep the polarizing movies in the dataset.

**3. How do you evaluate the effectiveness of these elements?**

- The effectiveness was determined by splitting the data into testing and validation datasets and using the accuracy measure to evaluate the performance of each model or technique being used. We also carried out 10-fold cross validation to further assess the performance across differents folds of data. We closely followed the difference between the testing performance and the baseline performance on individual tests as well as on each fold of the 10 fold cross validation in order to give us a better grasp of of the model performance on the kaggle testset.

**4. What have you tried that worked or didn’t?**

- We tried including more observations in the dataset and found that such an inclusion increased the time taken for the model to execute and did not improve model accuracy. During work on the model we came to support the view that including more points in this dataset is not guaranteed to improve performance.




### Subsampling

The cells below are used to identify the set of points to include in the training set, the training set focusses on the set of userid requiring predictions. The code in the cells below also includes checks on a selection of movies known to polarize opinion, as well as attempts to include more data in the dataset. The attempts to include more data and the check on a selection of movies has been commented out to speed code execution.

In [ ]:
if 'dfMtx' in vars(): del dfMtx   # release ~400MB RAM
dfK = df[df.rtg!=df.rtg]          # uid-mid-rtg for Kaggle submission (contains NaN ratings)

In [ ]:
dfT = df[df.rtg==df.rtg]
dfT = dfT[dfT.uid.isin(dfK.uid.unique())]  # all ratings on users requiring predictions (as adjacency list)

In [ ]:
#pd.set_option('display.max_rows', 500)
#dfMov
#df_find = dfK.loc[dfK['mid'] == 145]

In [ ]:
#dfT = dfT.loc[dfT['mid'] != 145]
#dfT = dfT.loc[dfT['mid'] != 54]
#dfT = dfT.loc[dfT['mid'] != 81]
#dfT = dfT.loc[dfT['mid'] != 98]
#dfT

In [ ]:
#df_find.astype(int).hist(density=True, alpha=0.3);

### Attempt to include more datapoints

In the cell below, we attempt to include more datapoints in the set of data used for training, yet we discovered that such an addition did not improve performance and caused the models to execute in a longer time. Therefore, the addition is not used and is commented out.

In [ ]:
#dfT = dfT.iloc[:4000000,:]
#dfT

## **Task 2. Modeling Approach**
Explain your modeling approach, i.e. ideas you tried and why you thought they would be helpful.

**1. How did these decision guide you in modeling?**

In order to explain our modeling approach , the steps below, highlight the elements we considered during model development.

- Using different models
- Hyperparameter tuning
- Evaluation using cross validation and accuracy

**2. How did you evaluate the effectiveness of these elements?**

**For using different models**: We tried different models from different algorithms with the Python Surprise library including SVD, SVDpp, NMF, CoClustering, KNNWithZScore, KNNWithMeans, and KNNBaseline. We found that the best peforming model is the SVDpp model. Please note that for this assignment we focussed on only the allowed models as per the rules of the competition and as such did not attempt any methods that fall outside the prescribed types: matrix decompositions, clustering, ensembles, and sklearn surprise.

**For hyperparameter tuning**: Amongst the different model algorithms we tried, we noted the some parameters were common, including the number of factors as per PCA and the number of epochs. We tried changing the factors from 2 to 50 and the number of epochs from 5 to 15 while monitoring the performance with cross validation. For the SVDpp model we also tried different learning rates from 0.005 to 0.2 and found good performance with the number of factors set at 10, number of epochs at 12 and the learning rate at 0.01 hinting that these choices are likely to be good choices for the model.

**For model evaluation**: We used cross validation to assess the performance of the models while we observed the accuracy. Such an approach enabled us to better grasp any performance improvements as a result of the models being selected. We also used the root mean square error to help us better understand the performance of the models.

**3. What else have you tried that worked or didn’t?**

During the development of the model, a few considerations came to mind and they are highlighted below:

- While using different models and assessing their performance performancem we noticed that the use of KNN models repeatedly caused the colab file to crash on memory usage despite reducing the number of points and altering the baseline options in the algorithm configuration. We reason that the relatively high dimensional nature of the data may influenced the capability of the KNN models to operate given the memory constraints of the colab file.
-While investigating the different sizes of data to include, we noticed that the act of focussing on only the userids known to require predictions caused an improvement in performance of the order aproximately 1%. Such a finding hints at the concept that at some stage in model development or training, including more points may not always imply better performance.

We also tried the following without improving our score:
Matrix factorization:
- We used the entire matrix for all experiments. We performed a grid search to find the best of filling the NaN values (mean, median, and multi-nominal sampling following each movie’s rating distribution) and the best number of components for the SVD.

PCA + Clustering :
- First, we used PCA to reduce the dimensionality. Then we tried to cluster by users, but it takes too long, so I decided to take each movie as a feature vector to find clusters, and depending on which cluster the movie is assigned to, we filled the NaN values with the cluster mean or median. We also performed Grid Search to find the best combination of components for the PCA and the number of clusters.

Collaborative filtering:
- We created a collaborative filtering algorithm that iteratively finds the best parameters (in this case, the parameter are just two matrixes U and V) to predict the original matrix.

We tried using KNNs to group the data and assign the rating to be the mean of the closest k neighbors.  We tried Euclidean Distance, Jaccardian distance [6] as well a custom metrics that combine the 2 (Euclidean distance * Jaccardian distance) in a pipeline to determine the neighbors.  Sklearns built in pdist couldn't work due to the memory allocations so we wrote our own sklearn pipeline model.  The thought behind it was that we use the jaccard (or in this case an abbreviated jaccard) to determing which users rated the same movies as the test subject, thinking that if they're watching the same movies they have the similar tastes looking at only users that rated that movie for the training.  Then we take the euclidean distance to see if they are giving similar ratings to the same movies and if they are their tastes are even more similar - so we take the mean of the top k closest people's score of the movie in question.  The performance despite hyperparameter tuning never got higher than 35%.

```python
# Failed distance metric
    def dist(self, u, v):
        non_nan_indices = ~np.isnan(u) & ~np.isnan(v)
        return np.linalg.norm(u[non_nan_indices] - v[non_nan_indices]) * (1 - len(non_nan_indices) / min(len(u), len(v)))
```

### Model Selection & Training

Multiple models were used with multiple hyperparameters for tuning. Out of the attempted models, the best performance was seen with the SVDpp algorithm. As such, the code for SVD, NMF, KNNWithZScore, KNNWithMeans, CoClustering, and KNNBaseline below is commented out. Further, hyperparameters chosen are n_factors (which was varied from 2 to 50) and the n_epochs (which was varied from 5 to 15), and the learning rate (which was varied from 0.005 to 0.2). Out of the selected paremeters, it was noted that performance does not improve past that seen at a learning rate of 0.01, n_epochs of 12 and n_factors of 10, thereby indicating that those choices are likely to be good candidates for model parameters.

Also include is the code used to train the models following the Dataset functionality from the Python Surprise library.

In [ ]:
import pandas as pd
from surprise import Dataset
from surprise import Reader
from surprise import SVD, SVDpp, NMF, KNNWithZScore, KNNWithMeans, CoClustering, KNNBaseline
from surprise.model_selection import train_test_split
from surprise.model_selection.split import KFold
from surprise import accuracy
from surprise.model_selection import cross_validate
from sklearn.metrics import accuracy_score
pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
%%time
reader = Reader(line_format = 'user item rating', rating_scale=(0,5))
data = Dataset.load_from_df(dfT,reader)
algo = SVDpp(n_factors = 10, random_state = 0, n_epochs = 12, lr_all = 0.010, cache_ratings = True)
# algo = SVD(n_factors = 2, random_state = 0, n_epochs = 11, lr_all = 0.012, biased = True)
# algo = NMF(n_factors = 10, random_state = 0, n_epochs = 13)
# algo = KNNBaseline(bsl_options = {"method": "als"})
# algo = KNNWithZScore()
# algo = KNNWithMeans()
# algo = CoClustering(n_cltr_u = 4, n_cltr_i = 4, random_state = 0, n_epochs = 10)
# cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

trainset, testset = train_test_split(data, test_size = .1, random_state = 0)

CPU times: user 3.69 s, sys: 104 ms, total: 3.79 s
Wall time: 5.47 s


In [ ]:
%%time
algo.fit(trainset)

CPU times: user 42 s, sys: 132 ms, total: 42.1 s
Wall time: 44.4 s


### Model Evaluation

The below cells show the accuracy measure to check the performance of the SVD++ model using training and validation data with cross validation. The evaluation was done multiple times, once with one training and one validation set, and once again with 10 fold cross validation.

In [ ]:
#%%time
#predictions = algo.test(testset)

In [ ]:
#%%time
#accuracy.rmse(predictions)

In [ ]:
#t_list = []
#r_list = []
#for i in range(len(predictions)):
#  t_list.append(predictions[i].r_ui)
#  r_list.append(predictions[i].est)
#r_list = pd.DataFrame(r_list).round(0)
#t_list = pd.DataFrame(t_list).round(0)

In [ ]:
#accuracy_score(t_list, r_list)

In [ ]:
#kf = KFold(n_splits=10, random_state=0)
#for trainset, testset in kf.split(data):
#  algo.fit(trainset)
#  print("completed fit")
#  predictions = algo.test(testset)
#  print("completed test")
#  t_list = []
#  r_list = []
#  for i in range(len(predictions)):
#    t_list.append(predictions[i].r_ui)
#    r_list.append(predictions[i].est)
#  r_list = pd.DataFrame(r_list).round(0)
#  t_list = pd.DataFrame(t_list).round(0)
#  print(accuracy_score(t_list, r_list))

In [ ]:
res_list = []
for i in range(len(dfK)):
  res_predict = algo.predict(dfK.uid[i],dfK.mid[i], clip = True)
  res_list.append(int(round(res_predict.est,0)))

In [ ]:
dfK['rtg'] = res_list

In [ ]:
dfK.index = (dfK.uid-1)*dfK.mid.max()+dfK.mid
dfK['rtg'] = dfK['rtg'].clip(1, 5)

In [ ]:
#_ = dfK.dropna(axis=0).astype(int).hist(density=True, alpha=0.3);

In [ ]:
# evaluate unconditional distributions of observed and predicted ratings. Learn from the differences to focus on wrong predictions
#plt.subplot(1, 1, 1)
#_ = r_list.dropna(axis=0).astype(int).hist(density=True, alpha=0.3);
#_ = r_list.astype(int).hist(density=True, alpha=0.3);
#_ = t_list.astype(int).hist(density=True, alpha=0.3);
#_ = plt.legend(['actual','predicted']);

In [ ]:
#dfX.iloc[:2,:30]   # original matrix with NaN's

In [ ]:
#dfX_est.loc[:2,:30]  # naive imputation with column means

In [ ]:
#%%time
#np.random.seed(0)
#for _ in range(5):                             # apply truncated SVD approximation a few times
#  U, D, Vt = svds(dfX_est.to_numpy(), k=10)               # find k eigenvectors (U,V) and eigenvalues (D)
#  dfX_est.loc[1:n, 1:p] = U @ np.diag(D) @ Vt  # k-dimensional estimate (as np array) of dfM (we expect ratings' noise to reduce)
#  mask = ~np.isnan(dfX)       # mask locations of NaN's
#  dfX_est[mask] = dfX[mask]   # place original/known ratings back into estimated uid×mid matrix
#dfX_est.loc[:2,:30]           # a "better" approximation of underlying ratings distribution (or so we hope ;) )

In [ ]:
## Submissions are scored on accuracy, not on continuous/regression error
#dfX_est_L = dfX_est.unstack().reset_index().round(0).astype(int).rename(columns={0:'rtg'})  # tall format with integer ratings
#dfK = dfX_est_L.merge(dfK, on=['mid','uid'], suffixes=['','_']).drop('rtg_', axis=1)
#dfK.index = (dfK.uid-1)*dfK.mid.max()+dfK.mid
#dfK['rtg'] = dfK['rtg'].clip(1, 5)  # restrict the range to 1,2,3,4,5 . Thanks to Yannis & Liz, See Piazza @67_f1
#dfK

In [ ]:
## evaluate unconditional distributions of observed and predicted ratings. Learn from the differences to focus on wrong predictions
#plt.subplot(1, 1, 1)
#_ = df.rtg.dropna(axis=0).astype(int).hist(density=True, alpha=0.3);
#_ = dfK.rtg.astype(int).hist(density=True, alpha=0.3);
#_ = plt.legend(['actual','predicted']);

In [ ]:
ToCSV(dfK[['rtg']].sort_index(), 'Group4Netflix')

# **References:**

[1] Géron, Aurélien. “Chapter 8.” Hands-on Machine Learning with Scikit-Learn, Keras, and Tensorflow: Concepts, Tools, and Techniques to Build Intelligent Systems, O’Reilly Media, Inc., Sebastopol, CA, 2019.

[2] "API Reference." Python Surprise, https://surprise.readthedocs.io/en/stable/getting_started.html. Accessed 30 June 2023.

[3] "API Reference." Scikit Accuracy Score, https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html. Accessed 30 June 2023.

[4] "API Refefence." Scikit. https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier. Accessed 23 June 2023.

[5] C. Thompson, "If You Liked This, You’re Sure to Love That", New York Times, https://www.nytimes.com/2008/11/23/magazine/23Netflix-t.html. Accessed 30 June 2023.

[6] Jaccard index. (2020, September 6). Wikipedia. https://en.wikipedia.org/wiki/Jaccard_index

<font size=5>⌛</font> <strong><font color=orange size=5>Do not exceed competition's runtime limit!</font></strong>

<hr color=red>


In [ ]:
tmr.ShowTime()    # measure Colab's runtime. Do not remove. Keep as the last cell in your notebook.

Runtime is 54 sec


## 💡**Starter Ideas**

1. Tune model **hyperparameters**.
  1. SVD's $k$ might be user-perceived movie genres (perhaps, an average user categorizes movies into just 5 or 50 genres) [🎦](https://youtu.be/sooj-_bXWgk)
1. Try smarter **sampling** from the training set to avoid using all observations (due to time constraint)
  1. Some movies may have consistent high/low ratings and do not distinguish user preferences (just add to runtime)
1. Try SVD on groups (say movie genres) with the largest impact on overall accuracy
1. Try smarter initial imputation, perhaps, conditioned on movie genres or other observed features
1. Try clustering techniques from [SKL](https://scikit-learn.org/stable/modules/clustering.html)
1. Try engineering new features using Movies' data (`movies.csv`)
1. Try supervised approach on clusters or engineered features
1. Try shift/scale ratings. Recall: SVD seeks largest quadratic error from mean without shifting first, like variance does. Since $1^2\ll5^2$, SVD might work better for higher ratings.
  1. Note: the baseline model squeezes predictions towards ratings 3 and 4, which is not the original distribution.
1. Try feature restriction, selection, and regularization (see ESL pp.140-141)
1. Do a thorough EDA. Are there movies that are liked by most viewers?
1. Try Robust PCA (RPC) [🎦](https://www.youtube.com/watch?v=yDpz0PqULXQ), [RPCA Github](https://github.com/dganguli/robust-pca)
1. Find polarized movies and tune for them (like "*Napoleon Dynamite*"). See [&#127910;](https://youtu.be/ImpV70uLxyw)
1. Try KNN [📄](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.143.596&rep=rep1&type=pdf), [📄](https://scholar.google.com/scholar?q=knn+netflix+prize)
1. Learn more methods to build/use latent representations [🎦](https://youtu.be/n3RKsY2H-NE),  [📄](https://ieeexplore.ieee.org/abstract/document/5197422), [📄](https://scholar.google.com/scholar?q=netflix+prize+challenge), [&#127910;](https://www.youtube.com/results?search_query=netflix+prize+svd), [📄](https://www.aaai.org/Papers/AAAI/2002/AAAI02-029.pdf),